In [14]:
import pandas as pd
import pickle

In [52]:
# Reading in data, basic cleaning 

df=pd.read_csv('../datafiles/stata.csv')

df = pd.get_dummies(df['CPO']).join(df)

del df['ID2']
del df['ReportedTotal']
del df['ReportedInd']
del df['Party']
del df['CPO']
df = df.fillna(0)
df.columns = df.columns.map(lambda x: x.lower())



In [68]:
df = pd.read_pickle('../datafiles/data.p')
a = pd.get_dummies(df['party']).rename(columns = {1: 'dem',
                                              2: 'rep'})
df = df.merge(a, left_index=True, right_index=True)
df = df.drop(['i', 'c', 'o', 'party', 'acceptpacs', 'seat' ], axis=1)

In [69]:
df['office']

0       FL26
1       MI01
2       CO03
3       PA15
4       NY24
5       NJ05
6       OR01
7       AK01
8       OH17
9       OH15
10      NC12
11      NH02
12      PA02
13      WA07
14      OH05
15      ID02
16      MD02
17      OH08
18      CA17
19      PA03
20      MO03
21      NY29
22      VA03
23      MA09
24      CO06
25      GA06
26      IA05
27      WA09
28      GA12
29      MD04
        ... 
2226    OH15
2227    NY08
2228    IL18
2229    CO02
2230    TX34
2231    OH07
2232    CA33
2233    ME01
2234    WA03
2235    OR05
2236    MO02
2237    OR05
2238    IN08
2239    FL15
2240    NM01
2241    TN01
2242    PA07
2243    AR02
2244    IN02
2245    AR04
2246    GA07
2247    KS03
2248    NH02
2249    NC11
2250    MI05
2251    NV01
2252    AL07
2253    RI01
2254    CA21
2255    MO02
Name: office, dtype: object

In [36]:
# Making new variables 

col_list_unitem= ['unitemind', 'unitempacs', 'unitemfor', 'unitemproxy']
col_list_item = ['itemind', 'unitempacs', 'unitemfor', 'unitemproxy']
df['unitemtotal'] = df[col_list_unitem].sum(axis=1) 
df['itemtotal'] = df[col_list_item].sum(axis=1)
df['total'] = df[['unitemtotal', 'itemtotal']].sum(axis=1)
df['unitemie'] = df[['unitemfor', 'unitemproxy']].sum(axis=1)
df['itemie'] = df[['itemfor', 'itemproxy']].sum(axis=1)
df['percunitem'] = df['unitemtotal']/df['total']
df['total'] = df['total'].apply(lambda x: float(x))
df['percpacs']= df['totalpacs']/df['total']
df['ie'] = df['iefor'] + df['ieproxy']
df['percie'] = df['ie']/df['total']
df['percind']= df['totalind']/df['total']
df['percstate']= df['instate']/df['total']

In [37]:
pickle.dump( df, open( "../datafiles/data.p", "wb" ) )